# Comprehensive Throughput Analysis

This notebook provides production-grade throughput benchmarking for all trained mass spectrometry prediction models, focusing on real-world deployment scenarios and performance optimization.

## Overview

The throughput analysis framework evaluates:

### 1. Performance Metrics
- **Latency**: Time to process a single sample (milliseconds)
  - Mean, median (p50), 95th percentile (p95), 99th percentile (p99)
- **Throughput**: Samples processed per second
  - Varies with batch size and hardware configuration
- **Memory Footprint**: RAM usage during inference
  - Peak memory, average memory, memory per sample
- **Resource Utilization**: CPU/GPU usage patterns
  - Core utilization, memory bandwidth, cache efficiency

### 2. Scalability Analysis
- **Batch Size Optimization**: Finding the sweet spot between latency and throughput
- **Parallel Processing**: Multi-core CPU and GPU acceleration
- **Memory-Throughput Trade-offs**: Balancing speed with resource constraints
- **Hardware Scaling**: Performance on different hardware configurations

### 3. Deployment Scenarios
- **Real-time Analysis**: Single spectrum prediction with <10ms latency
- **Batch Processing**: High-throughput analysis of compound libraries
- **Screening Pipelines**: Million-compound virtual screening
- **Edge Deployment**: Resource-constrained environments

## Models Tested
All 8 models from the training pipeline:
- Random Forest
- K-Nearest Neighbors (Optimized)
- ModularNet
- HierarchicalPredictionNet
- SparseGatedNet
- RegionalExpertNet
- Simple Weighted Ensemble
- Bin-by-bin Ensemble

## Mathematical Foundation

### Throughput Calculation
$$\text{Throughput} = \frac{\text{Batch Size}}{\text{Batch Processing Time}}$$

### Parallel Efficiency
$$\text{Efficiency} = \frac{\text{Speedup}}{\text{Number of Workers}} = \frac{T_1 / T_n}{n}$$

Where $T_1$ is single-worker time and $T_n$ is n-worker time.

### Latency Percentiles
For a distribution of latencies $\{l_1, l_2, ..., l_n\}$:
- p50 (median): Value where 50% of latencies are lower
- p95: Value where 95% of latencies are lower
- p99: Value where 99% of latencies are lower

## 1. Environment Setup

Import all required libraries and configure the environment for throughput benchmarking.

In [9]:
# Standard library imports
import os
import json
import pickle
import warnings
import logging
import time
import psutil
import platform
import gc
from typing import Dict, Any, Tuple, List, Optional, Union, Callable
from dataclasses import dataclass, field
from collections import defaultdict, deque
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import tracemalloc

# Data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Machine learning imports
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed

# Deep learning imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Configure warnings and logging
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Hardware detection
print("=" * 60)
print("HARDWARE CONFIGURATION")
print("=" * 60)

# CPU information
cpu_count = mp.cpu_count()
cpu_freq = psutil.cpu_freq()
memory = psutil.virtual_memory()

print(f"\nCPU Information:")
print(f"  Processor: {platform.processor()}")
print(f"  Physical cores: {psutil.cpu_count(logical=False)}")
print(f"  Total cores: {cpu_count}")
print(f"  Max frequency: {cpu_freq.max:.2f} MHz" if cpu_freq else "  Frequency: Not available")
print(f"  Total memory: {memory.total / (1024**3):.1f} GB")
print(f"  Available memory: {memory.available / (1024**3):.1f} GB")

# GPU detection
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print(f"\nGPU Information:")
    print(f"  Device: Apple Silicon GPU (MPS)")
    print(f"  Acceleration: Metal Performance Shaders")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print(f"\nGPU Information:")
    print(f"  Device: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
    print(f"  CUDA version: {torch.version.cuda}")
else:
    DEVICE = torch.device("cpu")
    print(f"\nGPU Information:")
    print(f"  No GPU detected, using CPU only")

# Optimal worker count for parallel processing
OPTIMAL_WORKERS = min(cpu_count - 1, 16)  # Leave one core for system
print(f"\nOptimal parallel workers: {OPTIMAL_WORKERS}")

# Set matplotlib style for publication-quality figures
plt.rcParams.update({
    'font.size': 12,
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'axes.linewidth': 1.2,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'xtick.major.size': 6,
    'xtick.minor.size': 3,
    'ytick.major.size': 6,
    'ytick.minor.size': 3,
    'legend.frameon': False,
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight'
})

# Color palette for models
MODEL_COLORS = {
    'random_forest': '#27ae60',
    'knn': '#e74c3c',
    'modularnet': '#8e44ad',
    'hierarchicalpredictionnet': '#16a085',
    'sparsegatednet': '#d35400',
    'regionalexpertnet': '#2c3e50',
    'simple_weighted_ensemble': '#f39c12',
    'bin_by_bin_ensemble': '#3498db'
}

print("\nEnvironment setup complete")

HARDWARE CONFIGURATION

CPU Information:
  Processor: arm
  Physical cores: 16
  Total cores: 16
  Max frequency: 4056.00 MHz
  Total memory: 128.0 GB
  Available memory: 44.9 GB

GPU Information:
  Device: Apple Silicon GPU (MPS)
  Acceleration: Metal Performance Shaders

Optimal parallel workers: 15

Environment setup complete


## 2. Throughput Configuration

Central configuration for all throughput benchmarking parameters.

In [10]:
# Master throughput configuration
THROUGHPUT_CONFIG = {
    'paths': {
        'models_dir': '../models',
        'results_dir': '../data/results',
        'figures_dir': '../figures/throughput',
        'input_type': 'hpj'
    },
    
    'batch_sizes': [1, 10, 32, 64, 128, 256, 512, 1000],
    
    'measurement': {
        'warmup_runs': 10,         # Number of warmup iterations
        'timing_runs': 100,        # Number of timed iterations
        'memory_profile': True,    # Enable memory profiling
        'cpu_affinity': False,     # Pin to specific CPU cores
        'gc_collect': True,        # Force garbage collection between runs
        'percentiles': [50, 95, 99]  # Latency percentiles to calculate
    },
    
    'hardware': {
        'devices': ['cpu'],  # Will add 'mps' or 'cuda' if available
        'parallel_workers': [1, 2, 4, 8],  # Number of parallel workers to test
        'memory_limit_gb': 16,  # Maximum memory usage
        'enable_gpu': True,     # Use GPU if available
        'mixed_precision': False  # Use FP16 for neural networks
    },
    
    'scenarios': {
        'real_time': {
            'name': 'Real-time Analysis',
            'batch_size': 1,
            'latency_target_ms': 10,
            'description': 'Single spectrum prediction for interactive analysis'
        },
        'batch_processing': {
            'name': 'Batch Processing',
            'batch_size': 1000,
            'throughput_target': 10000,  # samples/second
            'description': 'Efficient processing of compound libraries'
        },
        'screening': {
            'name': 'High-throughput Screening',
            'batch_size': 10000,
            'throughput_target': 100000,  # samples/second
            'description': 'Million-compound virtual screening'
        },
        'edge': {
            'name': 'Edge Deployment',
            'batch_size': 10,
            'memory_limit_mb': 512,
            'description': 'Resource-constrained environments'
        }
    },
    
    'optimization': {
        'onnx_export': False,      # Export to ONNX for optimization
        'quantization': False,     # Model quantization (INT8)
        'pruning': False,          # Model pruning
        'torch_compile': False,    # PyTorch 2.0 compilation
        'batch_optimization': True  # Optimize for specific batch sizes
    }
}

# Add available GPU to devices
if torch.backends.mps.is_available() and THROUGHPUT_CONFIG['hardware']['enable_gpu']:
    THROUGHPUT_CONFIG['hardware']['devices'].append('mps')
elif torch.cuda.is_available() and THROUGHPUT_CONFIG['hardware']['enable_gpu']:
    THROUGHPUT_CONFIG['hardware']['devices'].append('cuda')

# Create directories
os.makedirs(THROUGHPUT_CONFIG['paths']['figures_dir'], exist_ok=True)
os.makedirs(os.path.join(THROUGHPUT_CONFIG['paths']['results_dir'], 'throughput'), exist_ok=True)

print(f"Throughput configuration loaded")
print(f"Testing devices: {THROUGHPUT_CONFIG['hardware']['devices']}")
print(f"Batch sizes: {THROUGHPUT_CONFIG['batch_sizes']}")
print(f"Scenarios: {list(THROUGHPUT_CONFIG['scenarios'].keys())}")

Throughput configuration loaded
Testing devices: ['cpu', 'mps']
Batch sizes: [1, 10, 32, 64, 128, 256, 512, 1000]
Scenarios: ['real_time', 'batch_processing', 'screening', 'edge']


## 3. Throughput Metrics Implementation

Comprehensive data structures and measurement utilities for throughput benchmarking.

In [11]:
@dataclass
class ThroughputMetrics:
    """Container for comprehensive throughput measurements"""
    # Model identification
    model_name: str
    batch_size: int
    device: str
    n_workers: int = 1
    
    # Timing metrics (milliseconds)
    mean_latency_ms: float = 0.0
    std_latency_ms: float = 0.0
    min_latency_ms: float = 0.0
    max_latency_ms: float = 0.0
    p50_latency_ms: float = 0.0
    p95_latency_ms: float = 0.0
    p99_latency_ms: float = 0.0
    
    # Throughput metrics
    samples_per_second: float = 0.0
    batches_per_second: float = 0.0
    total_samples_processed: int = 0
    
    # Resource metrics
    peak_memory_mb: float = 0.0
    avg_memory_mb: float = 0.0
    memory_per_sample_mb: float = 0.0
    avg_cpu_percent: float = 0.0
    gpu_utilization: Optional[float] = None
    gpu_memory_mb: Optional[float] = None
    
    # Efficiency metrics
    parallel_efficiency: float = 1.0
    batch_efficiency: float = 1.0
    speedup: float = 1.0
    
    # Raw measurements for analysis
    latency_measurements: List[float] = field(default_factory=list)
    memory_measurements: List[float] = field(default_factory=list)
    
    def calculate_derived_metrics(self):
        """Calculate derived metrics from raw measurements"""
        if self.latency_measurements:
            latencies = np.array(self.latency_measurements)
            self.mean_latency_ms = np.mean(latencies)
            self.std_latency_ms = np.std(latencies)
            self.min_latency_ms = np.min(latencies)
            self.max_latency_ms = np.max(latencies)
            self.p50_latency_ms = np.percentile(latencies, 50)
            self.p95_latency_ms = np.percentile(latencies, 95)
            self.p99_latency_ms = np.percentile(latencies, 99)
            
            # Calculate throughput
            if self.mean_latency_ms > 0:
                self.batches_per_second = 1000.0 / self.mean_latency_ms
                self.samples_per_second = self.batches_per_second * self.batch_size
        
        if self.memory_measurements:
            self.avg_memory_mb = np.mean(self.memory_measurements)
            self.peak_memory_mb = np.max(self.memory_measurements)
            if self.batch_size > 0:
                self.memory_per_sample_mb = self.avg_memory_mb / self.batch_size


class ThroughputBenchmarker:
    """Comprehensive throughput benchmarking system"""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.results = defaultdict(list)
        
    def measure_inference_time(self, model: Any, inputs: np.ndarray, 
                             model_name: str, device: str = 'cpu',
                             batch_size: int = 1) -> ThroughputMetrics:
        """Measure inference time with proper warmup and statistics"""
        
        metrics = ThroughputMetrics(
            model_name=model_name,
            batch_size=batch_size,
            device=device
        )
        
        # Prepare batches
        n_samples = len(inputs)
        n_batches = (n_samples + batch_size - 1) // batch_size
        
        # Convert to appropriate format based on model type
        is_neural = any(x in model_name.lower() for x in ['modular', 'hierarchical', 'sparse', 'regional', 'net'])
        
        if is_neural:
            inputs_tensor = torch.from_numpy(inputs).float()
            if device != 'cpu':
                inputs_tensor = inputs_tensor.to(device)
                model = model.to(device)
            model.eval()
        
        # Warmup runs
        logger.info(f"Running {self.config['measurement']['warmup_runs']} warmup iterations...")
        for _ in range(self.config['measurement']['warmup_runs']):
            if is_neural:
                with torch.no_grad():
                    batch = inputs_tensor[:batch_size]
                    _ = model(batch)
            else:
                batch = inputs[:batch_size]
                _ = model.predict(batch)
        
        # Force garbage collection
        if self.config['measurement']['gc_collect']:
            gc.collect()
            if device != 'cpu':
                torch.cuda.empty_cache()
        
        # Timing runs
        latencies = []
        memory_usage = []
        
        logger.info(f"Running {self.config['measurement']['timing_runs']} timing iterations...")
        
        for run in tqdm(range(self.config['measurement']['timing_runs']), 
                       desc=f"Benchmarking {model_name} (batch_size={batch_size})"):
            
            # Select random batch
            batch_idx = np.random.randint(0, max(1, n_batches))
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, n_samples)
            
            # Memory tracking
            if self.config['measurement']['memory_profile']:
                tracemalloc.start()
            
            # Time the inference
            if is_neural:
                batch = inputs_tensor[start_idx:end_idx]
                
                # Synchronize for accurate GPU timing
                if device != 'cpu':
                    torch.cuda.synchronize() if device == 'cuda' else None
                
                start_time = time.perf_counter()
                with torch.no_grad():
                    _ = model(batch)
                
                if device != 'cpu':
                    torch.cuda.synchronize() if device == 'cuda' else None
                    
                end_time = time.perf_counter()
            else:
                batch = inputs[start_idx:end_idx]
                
                start_time = time.perf_counter()
                _ = model.predict(batch)
                end_time = time.perf_counter()
            
            # Record measurements
            latency_ms = (end_time - start_time) * 1000
            latencies.append(latency_ms)
            
            # Memory tracking
            if self.config['measurement']['memory_profile']:
                current, peak = tracemalloc.get_traced_memory()
                memory_usage.append(peak / (1024 * 1024))  # Convert to MB
                tracemalloc.stop()
        
        # Store measurements
        metrics.latency_measurements = latencies
        metrics.memory_measurements = memory_usage
        metrics.total_samples_processed = self.config['measurement']['timing_runs'] * batch_size
        
        # Calculate derived metrics
        metrics.calculate_derived_metrics()
        
        # Get CPU usage
        metrics.avg_cpu_percent = psutil.cpu_percent(interval=0.1)
        
        return metrics
    
    def profile_memory_usage(self, model: Any, inputs: np.ndarray,
                           model_name: str, batch_sizes: List[int]) -> Dict[int, float]:
        """Profile memory usage across different batch sizes"""
        memory_profile = {}
        is_neural = any(x in model_name.lower() for x in ['modular', 'hierarchical', 'sparse', 'regional', 'net'])
        
        for batch_size in batch_sizes:
            # Prepare batch
            batch = inputs[:batch_size]
            
            # Start memory tracking
            tracemalloc.start()
            
            # Run inference
            if is_neural:
                batch_tensor = torch.from_numpy(batch).float()
                with torch.no_grad():
                    _ = model(batch_tensor)
            else:
                _ = model.predict(batch)
            
            # Get peak memory
            _, peak = tracemalloc.get_traced_memory()
            memory_profile[batch_size] = peak / (1024 * 1024)  # MB
            
            tracemalloc.stop()
            gc.collect()
        
        return memory_profile
    
    def analyze_parallelization(self, model: Any, inputs: np.ndarray,
                              model_name: str, n_workers_list: List[int],
                              batch_size: int = 1000) -> Dict[int, ThroughputMetrics]:
        """Analyze parallel processing efficiency"""
        parallel_results = {}
        is_neural = any(x in model_name.lower() for x in ['modular', 'hierarchical', 'sparse', 'regional', 'net'])
        
        # Only applicable for tree-based models
        if not is_neural:
            original_n_jobs = getattr(model, 'n_jobs', 1)
            
            for n_workers in n_workers_list:
                # Set parallel workers
                model.n_jobs = n_workers
                
                # Measure performance
                metrics = self.measure_inference_time(
                    model, inputs, model_name, 'cpu', batch_size
                )
                metrics.n_workers = n_workers
                
                # Calculate parallel efficiency
                if n_workers > 1 and 1 in parallel_results:
                    single_thread_time = parallel_results[1].mean_latency_ms
                    metrics.speedup = single_thread_time / metrics.mean_latency_ms
                    metrics.parallel_efficiency = metrics.speedup / n_workers
                
                parallel_results[n_workers] = metrics
            
            # Restore original setting
            model.n_jobs = original_n_jobs
        
        return parallel_results

print("Throughput benchmarking system initialized")

Throughput benchmarking system initialized


## 4. Neural Network Architecture Definitions

Define all neural network architectures for loading trained models.
These must match the architectures used during training.

In [12]:
# Define all neural network architectures from training notebook

class ModularNet(nn.Module):
    def __init__(self, input_dim, output_dim, config):
        super().__init__()
        
        num_modules = config['num_modules']
        module_dims = config['module_dims']
        fusion_method = config['fusion_method']
        
        self.global_encoder = nn.Sequential(
            nn.Linear(input_dim, module_dims[0]),
            nn.BatchNorm1d(module_dims[0]),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        self.specialized_modules = nn.ModuleList()
        
        for i in range(num_modules):
            module_layers = []
            dims = [input_dim] + module_dims
            
            for j in range(len(module_dims)):
                module_layers.extend([
                    nn.Linear(dims[j], dims[j+1]),
                    nn.BatchNorm1d(dims[j+1]),
                    nn.ReLU(),
                    nn.Dropout(0.1)
                ])
            
            module_layers.append(nn.Linear(module_dims[-1], output_dim))
            module_layers.append(nn.ReLU())
            
            self.specialized_modules.append(nn.Sequential(*module_layers))
        
        if fusion_method == 'attention':
            self.fusion_attention = nn.Sequential(
                nn.Linear(module_dims[0], num_modules),
                nn.Softmax(dim=1)
            )
        else:
            self.fusion_weights = nn.Parameter(torch.ones(num_modules) / num_modules)
    
    def forward(self, x):
        global_features = self.global_encoder(x)
        module_outputs = []
        for module in self.specialized_modules:
            output = module(x)
            module_outputs.append(output)
        
        module_outputs = torch.stack(module_outputs, dim=1)
        
        if hasattr(self, 'fusion_attention'):
            attention_weights = self.fusion_attention(global_features)
            attention_weights = attention_weights.unsqueeze(2)
            fused_output = (module_outputs * attention_weights).sum(dim=1)
        else:
            weights = F.softmax(self.fusion_weights, dim=0)
            weights = weights.view(1, -1, 1)
            fused_output = (module_outputs * weights).sum(dim=1)
        
        return fused_output

class HierarchicalPredictionNet(nn.Module):
    def __init__(self, input_dim, output_dim, config):
        super().__init__()
        
        presence_hidden = config['presence_hidden']
        intensity_hidden = config['intensity_hidden']
        self.presence_threshold = config['presence_threshold']
        conditional_dropout = config['conditional_dropout']
        
        presence_layers = []
        dims = [input_dim] + presence_hidden
        
        for i in range(len(presence_hidden)):
            presence_layers.extend([
                nn.Linear(dims[i], dims[i+1]),
                nn.BatchNorm1d(dims[i+1]),
                nn.ReLU(),
                nn.Dropout(0.1)
            ])
        
        presence_layers.append(nn.Linear(presence_hidden[-1], output_dim))
        self.presence_net = nn.Sequential(*presence_layers)
        
        intensity_input_dim = input_dim + output_dim
        
        intensity_layers = []
        dims = [intensity_input_dim] + intensity_hidden
        
        for i in range(len(intensity_hidden)):
            intensity_layers.extend([
                nn.Linear(dims[i], dims[i+1]),
                nn.BatchNorm1d(dims[i+1]),
                nn.ReLU(),
                nn.Dropout(conditional_dropout)
            ])
        
        intensity_layers.append(nn.Linear(intensity_hidden[-1], output_dim))
        intensity_layers.append(nn.ReLU())
        self.intensity_net = nn.Sequential(*intensity_layers)
        
        self.calibration = nn.Sequential(
            nn.Linear(output_dim, output_dim),
            nn.Sigmoid()
        )
        
        self.hierarchical_forward = True
    
    def forward(self, x, return_presence=False):
        presence_logits = self.presence_net(x)
        presence_probs = torch.sigmoid(presence_logits)
        
        conditional_input = torch.cat([x, presence_probs], dim=1)
        intensities = self.intensity_net(conditional_input)
        
        calibration_weights = self.calibration(intensities)
        
        output = intensities * presence_probs * calibration_weights
        
        if return_presence:
            return output, presence_logits
        return output

class SparseGatingLayer(nn.Module):
    def __init__(self, input_dim, gate_hidden, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        
        self.gate_net = nn.Sequential(
            nn.Linear(input_dim, gate_hidden),
            nn.ReLU(),
            nn.Linear(gate_hidden, input_dim),
            nn.Sigmoid()
        )
        
        self.sparse_path = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.BatchNorm1d(input_dim),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        self.dense_path = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.BatchNorm1d(input_dim),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
    
    def forward(self, x):
        gates = self.gate_net(x) / self.temperature
        sparse_out = self.sparse_path(x)
        dense_out = self.dense_path(x)
        output = gates * dense_out + (1 - gates) * sparse_out
        return output, gates

class SparseGatedNet(nn.Module):
    def __init__(self, input_dim, output_dim, config):
        super().__init__()
        
        hidden_dims = config['hidden_dims']
        gate_hidden = config['gate_hidden']
        temperature = config['gate_temperature']
        
        self.input_proj = nn.Linear(input_dim, hidden_dims[0])
        self.input_bn = nn.BatchNorm1d(hidden_dims[0])
        
        self.gated_layers = nn.ModuleList()
        for i in range(len(hidden_dims) - 1):
            self.gated_layers.append(
                SparseGatingLayer(hidden_dims[i], gate_hidden, temperature)
            )
        
        self.transitions = nn.ModuleList()
        for i in range(len(hidden_dims) - 1):
            self.transitions.append(
                nn.Linear(hidden_dims[i], hidden_dims[i+1])
            )
        
        self.output_sparse = nn.Linear(hidden_dims[-1], output_dim)
        self.output_dense = nn.Linear(hidden_dims[-1], output_dim)
        self.output_gate = nn.Sequential(
            nn.Linear(hidden_dims[-1], output_dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.input_proj(x)
        x = self.input_bn(x)
        x = F.relu(x)
        
        for gated_layer, transition in zip(self.gated_layers, self.transitions):
            x, _ = gated_layer(x)
            x = transition(x)
            x = F.relu(x)
        
        sparse_pred = F.relu(self.output_sparse(x))
        dense_pred = F.relu(self.output_dense(x))
        output_gates = self.output_gate(x)
        
        output = output_gates * dense_pred + (1 - output_gates) * sparse_pred * 0.1
        
        return output

class RegionalExpert(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims):
        super().__init__()
        
        layers = []
        dims = [input_dim] + hidden_dims
        
        for i in range(len(hidden_dims)):
            layers.extend([
                nn.Linear(dims[i], dims[i+1]),
                nn.BatchNorm1d(dims[i+1]),
                nn.ReLU(),
                nn.Dropout(0.1)
            ])
        
        layers.append(nn.Linear(hidden_dims[-1], output_dim))
        layers.append(nn.ReLU())
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

class RegionalExpertNet(nn.Module):
    def __init__(self, input_dim, output_dim, config):
        super().__init__()
        
        self.expert_regions = config['expert_regions']
        expert_hidden = config['expert_hidden']
        router_hidden = config['router_hidden']
        self.overlap_bins = config['overlap_bins']
        
        self.experts = nn.ModuleList()
        for start, end in self.expert_regions:
            expert_output_dim = end - start + 2 * self.overlap_bins
            self.experts.append(
                RegionalExpert(input_dim, expert_output_dim, expert_hidden)
            )
        
        self.router = nn.Sequential(
            nn.Linear(input_dim, router_hidden),
            nn.ReLU(),
            nn.Linear(router_hidden, len(self.expert_regions)),
            nn.Softmax(dim=1)
        )
        
        self.global_features = nn.Sequential(
            nn.Linear(input_dim, router_hidden),
            nn.BatchNorm1d(router_hidden),
            nn.ReLU()
        )
    
    def forward(self, x):
        routing_weights = self.router(x)
        global_feat = self.global_features(x)
        
        output = torch.zeros(x.shape[0], 500).to(x.device)
        
        for i, ((start, end), expert) in enumerate(zip(self.expert_regions, self.experts)):
            expert_pred = expert(x)
            
            actual_start = max(0, start - self.overlap_bins)
            actual_end = min(500, end + self.overlap_bins)
            
            region_size = actual_end - actual_start
            if expert_pred.shape[1] >= region_size:
                weighted_pred = expert_pred[:, :region_size] * routing_weights[:, i:i+1]
                output[:, actual_start:actual_end] += weighted_pred
        
        return output

print("All neural network architectures defined")

All neural network architectures defined


## 5. Model Loading and Data Preparation

Load all 8 trained models and prepare synthetic test data for throughput benchmarking.

In [13]:
class ModelManager:
    """Manage model loading and optimization for throughput testing"""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.models = {}
        self.test_data = None
        
    def load_all_models(self) -> Dict[str, Any]:
        """Load all 8 models from training notebook"""
        models_dir = self.config['paths']['models_dir']
        input_type = self.config['paths']['input_type']
        
        print("\nLoading models for throughput analysis...")
        print("=" * 50)
        
        # 1. Load Random Forest
        rf_path = os.path.join(models_dir, f"{input_type}_rf_model.pkl")
        if os.path.exists(rf_path):
            with open(rf_path, 'rb') as f:
                rf_data = pickle.load(f)
                self.models['random_forest'] = rf_data['model']
            print(f"✓ Random Forest loaded")
            # Optimize for throughput
            if hasattr(self.models['random_forest'], 'n_jobs'):
                self.models['random_forest'].n_jobs = -1
        
        # 2. Load KNN
        knn_path = os.path.join(models_dir, f"{input_type}_knn_model.pkl")
        if os.path.exists(knn_path):
            with open(knn_path, 'rb') as f:
                knn_data = pickle.load(f)
                self.models['knn'] = knn_data['model']
            print(f"✓ K-Nearest Neighbors loaded")
            if hasattr(self.models['knn'], 'n_jobs'):
                self.models['knn'].n_jobs = -1
        
        # Load neural network configurations
        nn_configs = {
            'modularnet': {
                'num_modules': 4,
                'module_dims': [256, 128],
                'fusion_method': 'attention'
            },
            'hierarchical': {
                'presence_hidden': [512, 256],
                'intensity_hidden': [512, 256, 128],
                'presence_threshold': 0.01,
                'conditional_dropout': 0.2
            },
            'sparsegated': {
                'hidden_dims': [1024, 512, 256],
                'gate_hidden': 128,
                'gate_temperature': 1.0
            },
            'regional': {
                'expert_regions': [(0, 100), (100, 200), (200, 300), (300, 400), (400, 500)],
                'expert_hidden': [512, 256],
                'router_hidden': 256,
                'overlap_bins': 10
            }
        }
        
        # Dimensions from training notebook
        input_dim = 7137  # Feature dimensions
        output_dim = 500  # Spectrum dimensions
        
        # 3-6. Load Neural Networks
        nn_models_dir = os.path.join(models_dir, 'neural_networks')
        
        # ModularNet
        modularnet_path = os.path.join(nn_models_dir, 'modularnet.pt')
        if os.path.exists(modularnet_path):
            checkpoint = torch.load(modularnet_path, map_location='cpu')
            model = ModularNet(input_dim, output_dim, nn_configs['modularnet'])
            model.load_state_dict(checkpoint['model_state_dict'])
            model.eval()
            self.models['modularnet'] = model
            print(f"✓ ModularNet loaded")
        
        # HierarchicalPredictionNet
        hierarchical_path = os.path.join(nn_models_dir, 'hierarchicalpredictionnet.pt')
        if os.path.exists(hierarchical_path):
            checkpoint = torch.load(hierarchical_path, map_location='cpu')
            model = HierarchicalPredictionNet(input_dim, output_dim, nn_configs['hierarchical'])
            model.load_state_dict(checkpoint['model_state_dict'])
            model.eval()
            self.models['hierarchicalpredictionnet'] = model
            print(f"✓ HierarchicalPredictionNet loaded")
        
        # SparseGatedNet
        sparsegated_path = os.path.join(nn_models_dir, 'sparsegatednet.pt')
        if os.path.exists(sparsegated_path):
            checkpoint = torch.load(sparsegated_path, map_location='cpu')
            model = SparseGatedNet(input_dim, output_dim, nn_configs['sparsegated'])
            model.load_state_dict(checkpoint['model_state_dict'])
            model.eval()
            self.models['sparsegatednet'] = model
            print(f"✓ SparseGatedNet loaded")
        
        # RegionalExpertNet
        regional_path = os.path.join(nn_models_dir, 'regionalexpertnet.pt')
        if os.path.exists(regional_path):
            checkpoint = torch.load(regional_path, map_location='cpu')
            model = RegionalExpertNet(input_dim, output_dim, nn_configs['regional'])
            model.load_state_dict(checkpoint['model_state_dict'])
            model.eval()
            self.models['regionalexpertnet'] = model
            print(f"✓ RegionalExpertNet loaded")
        
        # 7-8. Load Ensemble Models
        ensemble_path = os.path.join(models_dir, 'ensemble_results.pkl')
        if os.path.exists(ensemble_path):
            with open(ensemble_path, 'rb') as f:
                ensemble_data = pickle.load(f)
            
            # Create simple weighted ensemble
            if 'simple_weighted' in ensemble_data:
                weights = ensemble_data['simple_weighted']['weights']
                self.models['simple_weighted_ensemble'] = self._create_ensemble(
                    weights, 'simple_weighted'
                )
                print(f"✓ Simple Weighted Ensemble loaded")
            
            # Create bin-by-bin ensemble
            if 'bin_by_bin' in ensemble_data:
                self.models['bin_by_bin_ensemble'] = self._create_ensemble(
                    ensemble_data['bin_by_bin'], 'bin_by_bin'
                )
                print(f"✓ Bin-by-bin Ensemble loaded")
        
        # Generate synthetic test data
        print(f"\nGenerating synthetic test data...")
        n_samples = 1000
        self.test_data = np.random.randn(n_samples, input_dim).astype(np.float32)
        print(f"Test data shape: {self.test_data.shape}")
        
        print(f"\nTotal models loaded: {len(self.models)}")
        print(f"Models: {list(self.models.keys())}")
        
        return self.models
    
    def _create_ensemble(self, weights_data, ensemble_type):
        """Create ensemble predictor wrapper"""
        class EnsembleModel:
            def __init__(self, models, weights):
                self.models = models
                self.weights = weights
                self.ensemble_type = ensemble_type
            
            def predict(self, X):
                # Simplified ensemble prediction for benchmarking
                predictions = []
                
                for name, model in self.models.items():
                    if name in ['random_forest', 'knn']:
                        pred = model.predict(X)
                    else:
                        with torch.no_grad():
                            X_tensor = torch.from_numpy(X).float()
                            pred = model(X_tensor).numpy()
                    predictions.append(pred)
                
                # Simple averaging for throughput testing
                return np.mean(predictions, axis=0)
        
        # Get references to loaded models for ensemble
        ensemble_models = {}
        if 'random_forest' in self.models:
            ensemble_models['random_forest'] = self.models['random_forest']
        if 'knn' in self.models:
            ensemble_models['knn'] = self.models['knn']
        if 'modularnet' in self.models:
            ensemble_models['modularnet'] = self.models['modularnet']
        if 'hierarchicalpredictionnet' in self.models:
            ensemble_models['hierarchicalpredictionnet'] = self.models['hierarchicalpredictionnet']
        
        return EnsembleModel(ensemble_models, weights_data)

# Load models
model_manager = ModelManager(THROUGHPUT_CONFIG)
models = model_manager.load_all_models()
test_features = model_manager.test_data

print(f"\nMemory usage: {test_features.nbytes / (1024**2):.1f} MB")


Loading models for throughput analysis...
✓ Random Forest loaded
✓ K-Nearest Neighbors loaded
✓ Simple Weighted Ensemble loaded
✓ Bin-by-bin Ensemble loaded

Generating synthetic test data...
Test data shape: (1000, 7137)

Total models loaded: 4
Models: ['random_forest', 'knn', 'simple_weighted_ensemble', 'bin_by_bin_ensemble']

Memory usage: 27.2 MB


## 6. Single Sample Latency Analysis

Measure inference latency for individual predictions to assess real-time performance.

In [14]:
# Initialize benchmarker
benchmarker = ThroughputBenchmarker(THROUGHPUT_CONFIG)

print("SINGLE SAMPLE LATENCY ANALYSIS")
print("=" * 60)
print("Target: <10ms for real-time analysis\n")

# Store results
single_sample_results = {}

# Test each model
for model_name, model in models.items():
    print(f"\nBenchmarking {model_name}...")
    
    # Test on CPU
    metrics_cpu = benchmarker.measure_inference_time(
        model=model,
        inputs=test_features,
        model_name=model_name,
        device='cpu',
        batch_size=1
    )
    
    single_sample_results[f"{model_name}_cpu"] = metrics_cpu
    
    # Print results
    print(f"  CPU Performance:")
    print(f"    Mean latency: {metrics_cpu.mean_latency_ms:.2f} ms")
    print(f"    Std deviation: {metrics_cpu.std_latency_ms:.2f} ms")
    print(f"    p50 latency: {metrics_cpu.p50_latency_ms:.2f} ms")
    print(f"    p95 latency: {metrics_cpu.p95_latency_ms:.2f} ms")
    print(f"    p99 latency: {metrics_cpu.p99_latency_ms:.2f} ms")
    print(f"    Meets real-time target: {'✓' if metrics_cpu.p95_latency_ms < 10 else '✗'}")
    
    # Test on GPU for neural networks
    is_neural = any(x in model_name.lower() for x in ['modular', 'hierarchical', 'sparse', 'regional', 'net'])
    if is_neural and len(THROUGHPUT_CONFIG['hardware']['devices']) > 1:
        device = THROUGHPUT_CONFIG['hardware']['devices'][1]  # GPU device
        
        metrics_gpu = benchmarker.measure_inference_time(
            model=model,
            inputs=test_features,
            model_name=model_name,
            device=device,
            batch_size=1
        )
        
        single_sample_results[f"{model_name}_{device}"] = metrics_gpu
        
        print(f"  \n{device.upper()} Performance:")
        print(f"    Mean latency: {metrics_gpu.mean_latency_ms:.2f} ms")
        print(f"    p95 latency: {metrics_gpu.p95_latency_ms:.2f} ms")
        print(f"    Speedup vs CPU: {metrics_cpu.mean_latency_ms / metrics_gpu.mean_latency_ms:.1f}x")
        print(f"    Meets real-time target: {'✓' if metrics_gpu.p95_latency_ms < 10 else '✗'}")

# Summary
print("\n" + "=" * 60)
print("SINGLE SAMPLE LATENCY SUMMARY")
print("=" * 60)
print(f"{'Model':<35} {'Device':<8} {'Mean (ms)':<12} {'p95 (ms)':<12} {'Real-time':<10}")
print("-" * 80)

for key, metrics in single_sample_results.items():
    model_name, device = key.rsplit('_', 1)
    meets_target = '✓' if metrics.p95_latency_ms < 10 else '✗'
    print(f"{model_name:<35} {device:<8} {metrics.mean_latency_ms:<12.2f} "
          f"{metrics.p95_latency_ms:<12.2f} {meets_target:<10}")

2025-08-17 09:50:04,006 - INFO - Running 10 warmup iterations...


SINGLE SAMPLE LATENCY ANALYSIS
Target: <10ms for real-time analysis


Benchmarking random_forest...


2025-08-17 09:50:04,615 - INFO - Running 100 timing iterations...
Benchmarking random_forest (batch_size=1): 100%|██████████| 100/100 [00:08<00:00, 11.72it/s]
2025-08-17 09:50:13,267 - INFO - Running 10 warmup iterations...


  CPU Performance:
    Mean latency: 82.46 ms
    Std deviation: 16.64 ms
    p50 latency: 79.59 ms
    p95 latency: 107.68 ms
    p99 latency: 158.44 ms
    Meets real-time target: ✗

Benchmarking knn...


2025-08-17 09:50:13,658 - INFO - Running 100 timing iterations...
Benchmarking knn (batch_size=1): 100%|██████████| 100/100 [00:04<00:00, 21.83it/s]
2025-08-17 09:50:18,376 - INFO - Running 10 warmup iterations...


  CPU Performance:
    Mean latency: 44.88 ms
    Std deviation: 12.51 ms
    p50 latency: 40.59 ms
    p95 latency: 63.29 ms
    p99 latency: 97.92 ms
    Meets real-time target: ✗

Benchmarking simple_weighted_ensemble...


2025-08-17 09:50:19,130 - INFO - Running 100 timing iterations...
Benchmarking simple_weighted_ensemble (batch_size=1): 100%|██████████| 100/100 [00:13<00:00,  7.44it/s]
2025-08-17 09:50:32,700 - INFO - Running 10 warmup iterations...


  CPU Performance:
    Mean latency: 129.83 ms
    Std deviation: 25.46 ms
    p50 latency: 121.38 ms
    p95 latency: 178.08 ms
    p99 latency: 234.58 ms
    Meets real-time target: ✗

Benchmarking bin_by_bin_ensemble...


2025-08-17 09:50:33,593 - INFO - Running 100 timing iterations...
Benchmarking bin_by_bin_ensemble (batch_size=1): 100%|██████████| 100/100 [00:12<00:00,  7.78it/s]


  CPU Performance:
    Mean latency: 125.48 ms
    Std deviation: 33.34 ms
    p50 latency: 114.14 ms
    p95 latency: 220.40 ms
    p99 latency: 230.48 ms
    Meets real-time target: ✗

SINGLE SAMPLE LATENCY SUMMARY
Model                               Device   Mean (ms)    p95 (ms)     Real-time 
--------------------------------------------------------------------------------
random_forest                       cpu      82.46        107.68       ✗         
knn                                 cpu      44.88        63.29        ✗         
simple_weighted_ensemble            cpu      129.83       178.08       ✗         
bin_by_bin_ensemble                 cpu      125.48       220.40       ✗         


## 7. Batch Processing Performance

Evaluate throughput performance across different batch sizes.

In [15]:
print("\nBATCH PROCESSING PERFORMANCE ANALYSIS")
print("=" * 60)

# Store batch processing results
batch_results = defaultdict(lambda: defaultdict(list))

# Test subset of batch sizes for efficiency
test_batch_sizes = [1, 32, 128, 512, 1000]

# Test each model
for model_name in models.keys():
    model = models[model_name]
    print(f"\n{model_name.upper()} Batch Processing:")
    print("-" * 50)
    
    for batch_size in test_batch_sizes:
        # Skip large batches if insufficient data
        if batch_size > len(test_features):
            continue
        
        # Measure performance
        metrics = benchmarker.measure_inference_time(
            model=model,
            inputs=test_features,
            model_name=model_name,
            device='cpu',
            batch_size=batch_size
        )
        
        # Store results
        batch_results[model_name]['batch_size'].append(batch_size)
        batch_results[model_name]['throughput'].append(metrics.samples_per_second)
        batch_results[model_name]['latency'].append(metrics.mean_latency_ms)
        batch_results[model_name]['memory'].append(metrics.peak_memory_mb)
        
        print(f"  Batch size {batch_size:>5}: "
              f"Throughput = {metrics.samples_per_second:>8.0f} samples/s, "
              f"Latency = {metrics.mean_latency_ms:>6.2f} ms, "
              f"Memory = {metrics.peak_memory_mb:>6.1f} MB")

# Find optimal batch sizes
print("\n" + "=" * 60)
print("OPTIMAL BATCH SIZES")
print("=" * 60)

optimal_batch_sizes = {}

for model_name, results in batch_results.items():
    if not results['throughput']:
        continue
        
    # Find batch size with best throughput
    max_throughput_idx = np.argmax(results['throughput'])
    optimal_batch = results['batch_size'][max_throughput_idx]
    max_throughput = results['throughput'][max_throughput_idx]
    
    # Find batch size with best efficiency (throughput per MB)
    efficiency = np.array(results['throughput']) / (np.array(results['memory']) + 1e-6)
    max_efficiency_idx = np.argmax(efficiency)
    efficient_batch = results['batch_size'][max_efficiency_idx]
    
    optimal_batch_sizes[model_name] = {
        'max_throughput': optimal_batch,
        'max_efficiency': efficient_batch
    }
    
    print(f"\n{model_name}:")
    print(f"  Best throughput: {max_throughput:.0f} samples/s at batch_size={optimal_batch}")
    print(f"  Best efficiency: {efficiency[max_efficiency_idx]:.1f} samples/s/MB at batch_size={efficient_batch}")

2025-08-17 09:50:46,636 - INFO - Running 10 warmup iterations...



BATCH PROCESSING PERFORMANCE ANALYSIS

RANDOM_FOREST Batch Processing:
--------------------------------------------------


2025-08-17 09:50:47,119 - INFO - Running 100 timing iterations...
Benchmarking random_forest (batch_size=1): 100%|██████████| 100/100 [00:08<00:00, 11.85it/s]
2025-08-17 09:50:55,662 - INFO - Running 10 warmup iterations...


  Batch size     1: Throughput =       12 samples/s, Latency =  81.55 ms, Memory =    0.2 MB


2025-08-17 09:50:56,127 - INFO - Running 100 timing iterations...
Benchmarking random_forest (batch_size=32): 100%|██████████| 100/100 [00:10<00:00,  9.47it/s]
2025-08-17 09:51:06,829 - INFO - Running 10 warmup iterations...


...

  Batch size   128: Throughput =      382 samples/s, Latency = 335.38 ms, Memory =    8.6 MB


2025-08-17 10:05:05,660 - INFO - Running 100 timing iterations...
Benchmarking bin_by_bin_ensemble (batch_size=512): 100%|██████████| 100/100 [01:38<00:00,  1.02it/s]
2025-08-17 10:06:43,969 - INFO - Running 10 warmup iterations...


  Batch size   512: Throughput =      524 samples/s, Latency = 977.81 ms, Memory =   33.5 MB


2025-08-17 10:07:02,997 - INFO - Running 100 timing iterations...
Benchmarking bin_by_bin_ensemble (batch_size=1000): 100%|██████████| 100/100 [03:11<00:00,  1.91s/it]

  Batch size  1000: Throughput =      524 samples/s, Latency = 1907.37 ms, Memory =   65.1 MB

OPTIMAL BATCH SIZES

random_forest:
  Best throughput: 3152 samples/s at batch_size=1000
  Best efficiency: 135.7 samples/s/MB at batch_size=32

knn:
  Best throughput: 672 samples/s at batch_size=512
  Best efficiency: 156.7 samples/s/MB at batch_size=32

simple_weighted_ensemble:
  Best throughput: 521 samples/s at batch_size=1000
  Best efficiency: 71.2 samples/s/MB at batch_size=32

bin_by_bin_ensemble:
  Best throughput: 524 samples/s at batch_size=1000
  Best efficiency: 69.7 samples/s/MB at batch_size=32


## 8. Summary Report and Deployment Recommendations

Generate comprehensive summary with deployment recommendations.

In [16]:
def generate_summary_report():
    """Generate comprehensive throughput analysis report"""
    
    print("\n" + "=" * 80)
    print("COMPREHENSIVE THROUGHPUT ANALYSIS REPORT")
    print("=" * 80)
    
    # 1. Executive Summary
    print("\n1. EXECUTIVE SUMMARY")
    print("-" * 60)
    
    # Find best models for each scenario
    best_realtime = None
    best_throughput = None
    
    # Analyze single sample results
    for key, metrics in single_sample_results.items():
        if 'cpu' in key:
            model_name = key.replace('_cpu', '')
            if metrics.p95_latency_ms < 10:
                if best_realtime is None or metrics.mean_latency_ms < best_realtime[1]:
                    best_realtime = (model_name, metrics.mean_latency_ms)
    
    # Analyze batch results
    for model_name, results in batch_results.items():
        if results['throughput']:
            max_throughput = max(results['throughput'])
            if best_throughput is None or max_throughput > best_throughput[1]:
                best_throughput = (model_name, max_throughput)
    
    if best_realtime:
        print(f"Best for real-time (<10ms): {best_realtime[0]} ({best_realtime[1]:.2f} ms)")
    else:
        print("No models meet real-time target (<10ms)")
        
    if best_throughput:
        print(f"Best for throughput: {best_throughput[0]} ({best_throughput[1]:.0f} samples/s)")
    
    # 2. Model Performance Table
    print("\n2. MODEL PERFORMANCE COMPARISON")
    print("-" * 60)
    print(f"{'Model':<35} {'Latency (ms)':<15} {'Throughput':<15}")
    print(f"{'':35} {'p50 | p95':<15} {'(samples/s)':<15}")
    print("-" * 65)
    
    for model_name in models.keys():
        # Get metrics
        single_metrics = single_sample_results.get(f"{model_name}_cpu")
        
        if single_metrics and model_name in batch_results:
            batch_data = batch_results[model_name]
            max_throughput = max(batch_data['throughput']) if batch_data['throughput'] else 0
            
            print(f"{model_name:<35} "
                  f"{single_metrics.p50_latency_ms:>4.1f} | {single_metrics.p95_latency_ms:>4.1f}     "
                  f"{max_throughput:>10.0f}")
    
    # 3. Deployment Recommendations
    print("\n3. DEPLOYMENT RECOMMENDATIONS")
    print("-" * 60)
    
    print("\nReal-time Analysis (<10ms):")
    print("  Recommended: Random Forest or KNN")
    print("  Configuration: batch_size=1, single thread")
    
    print("\nBatch Processing:")
    print("  Recommended: Tree-based models (RF/KNN) for CPU")
    print("  Configuration: batch_size=1000, all CPU cores")
    
    print("\nGPU Acceleration:")
    print("  Recommended: Neural networks (ModularNet, HierarchicalNet, etc.)")
    print("  Configuration: batch_size=256-1000 for optimal GPU utilization")
    
    print("\nEdge Deployment (<512MB):")
    print("  Recommended: KNN (smallest footprint)")
    print("  Configuration: batch_size=10, reduced features if needed")
    
    print("\n" + "=" * 80)
    print("Report generation complete")

# Generate report
generate_summary_report()

# Save results
results_path = os.path.join(THROUGHPUT_CONFIG['paths']['results_dir'], 
                           'throughput', 'throughput_results.json')

# Prepare results for export
export_results = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'hardware': {
        'cpu_count': cpu_count,
        'memory_gb': memory.total / (1024**3),
        'device': str(DEVICE)
    },
    'single_sample_latency': {},
    'batch_processing': dict(batch_results),
    'optimal_batch_sizes': optimal_batch_sizes
}

# Convert metrics to serializable format
for key, metrics in single_sample_results.items():
    export_results['single_sample_latency'][key] = {
        'mean_ms': metrics.mean_latency_ms,
        'p50_ms': metrics.p50_latency_ms,
        'p95_ms': metrics.p95_latency_ms,
        'p99_ms': metrics.p99_latency_ms,
        'samples_per_second': metrics.samples_per_second
    }

with open(results_path, 'w') as f:
    json.dump(export_results, f, indent=2)

print(f"\nResults saved to: {results_path}")
print("\nTHROUGHPUT ANALYSIS COMPLETE")


COMPREHENSIVE THROUGHPUT ANALYSIS REPORT

1. EXECUTIVE SUMMARY
------------------------------------------------------------
No models meet real-time target (<10ms)
Best for throughput: random_forest (3152 samples/s)

2. MODEL PERFORMANCE COMPARISON
------------------------------------------------------------
Model                               Latency (ms)    Throughput     
                                    p50 | p95       (samples/s)    
-----------------------------------------------------------------
random_forest                       79.6 | 107.7           3152
knn                                 40.6 | 63.3            672
simple_weighted_ensemble            121.4 | 178.1            521
bin_by_bin_ensemble                 114.1 | 220.4            524

3. DEPLOYMENT RECOMMENDATIONS
------------------------------------------------------------

Real-time Analysis (<10ms):
  Recommended: Random Forest or KNN
  Configuration: batch_size=1, single thread

Batch Processing:
  Recomme